Downloading Data:
=================

- Client ID: sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9
- Client secret: BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5

Links:
- https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process/Examples/S2L2A.html#true-color
- https://documentation.dataspace.copernicus.eu/notebook-samples/sentinelhub/cloudless_process_api.html



In [13]:
# Print basic timestamp and python version information

import time
import sys

print("Timestamp: ", time.strftime("%Y-%m-%d %H:%M:%S"))
print("Python version: ", sys.version)

Timestamp:  2025-01-22 14:52:30
Python version:  3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)]


In [2]:
# Setup the environment with pip installs and imports

installs = ["sentinelhub", "numpy", "pandas", "matplotlib", "requests", "getpass"]
for package in installs:
    !pip install {package}

ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass


In [14]:


# Utilities
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)



In [39]:
# Retrieving your Access Token

def get_access_token(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Access token creation failed. Reponse from the server was: {r.json()}"
        )
    return r.json()["access_token"]


username = "davesteadman@me.com"
pwurd    = "Copper-Esa-3141"

access_token = get_access_token(username, pwurd)



In [29]:
# Basic data

start_date = "2022-06-01"
end_date = "2022-06-10"
data_collection = "SENTINEL-2"

aoi = "POLYGON((4.220581 50.958859,4.521264 50.953236,4.545977 50.906064,4.541858 50.802029,4.489685 50.763825,4.23843 50.767734,4.192435 50.806369,4.189689 50.907363,4.220581 50.958859))'"


json = requests.get(
    f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}) and ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T00:00:00.000Z"
).json()
pd.DataFrame.from_dict(json["value"]).head(5)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,bb412273-69a3-5afa-8d2e-774f63e1c568,S2B_MSIL1C_20220601T103629_N0400_R008_T31UFS_2...,application/octet-stream,0,2022-06-01T15:15:13.475000Z,2022-06-01T15:20:38.977256Z,2022-06-01T15:20:38.977256Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2022/06/01/S2B_MSIL...,[],"{'Start': '2022-06-01T10:36:29.024000Z', 'End'...",geography'SRID=4326;POLYGON ((4.4362457433229 ...,"{'type': 'Polygon', 'coordinates': [[[4.436245..."
1,application/octet-stream,f83b6e00-8669-5f6c-96e1-f8c82dd98909,S2B_MSIL1C_20220601T103629_N0400_R008_T31UES_2...,application/octet-stream,0,2022-06-01T15:17:31.386000Z,2022-06-01T15:22:27.713076Z,2022-06-01T15:22:27.713076Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2022/06/01/S2B_MSIL...,[],"{'Start': '2022-06-01T10:36:29.024000Z', 'End'...",geography'SRID=4326;POLYGON ((4.0716897587579 ...,"{'type': 'Polygon', 'coordinates': [[[4.071689..."
2,application/octet-stream,794333bd-1015-5a46-bd22-0f4b943897c5,S2B_MSIL2A_20220601T103629_N0400_R008_T31UES_2...,application/octet-stream,0,2022-06-01T17:21:07.927000Z,2022-06-01T17:26:25.056082Z,2022-06-01T17:26:25.056082Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/06/01/S2B_MSIL...,[],"{'Start': '2022-06-01T10:36:29.024000Z', 'End'...",geography'SRID=4326;POLYGON ((4.0716897587579 ...,"{'type': 'Polygon', 'coordinates': [[[4.071689..."
3,application/octet-stream,550c10a8-3a31-5f28-8d68-e0b7086dbc8f,S2B_MSIL2A_20220601T103629_N0400_R008_T31UFS_2...,application/octet-stream,0,2022-06-01T17:40:10.814000Z,2022-06-01T17:47:49.803842Z,2022-06-01T17:47:49.803842Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/06/01/S2B_MSIL...,[],"{'Start': '2022-06-01T10:36:29.024000Z', 'End'...",geography'SRID=4326;POLYGON ((4.4362457433229 ...,"{'type': 'Polygon', 'coordinates': [[[4.436245..."
4,application/octet-stream,08e47196-bb90-4c97-811f-d350fd01e4b5,S2B_MSIL2A_20220601T103629_N0510_R008_T31UES_2...,application/octet-stream,252868035,2024-10-23T16:39:07.432000Z,2024-12-16T17:33:05.054519Z,2024-12-16T17:33:11.153529Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A_N0500/2022/06/01/S2...,"[{'Value': '7930f5aa483152543b9597081a51c111',...","{'Start': '2022-06-01T10:36:29.024000Z', 'End'...",geography'SRID=4326;POLYGON ((4.07166873560485...,"{'type': 'Polygon', 'coordinates': [[[4.071668..."


In [12]:
maxlat = 53
minlat = 52
minlon = -3
maxlon = -2

start_date      = "2023-12-01"
end_date        = "2024-05-10"
data_collection = "SENTINEL-2"

aoi             = f"POLYGON(({minlon} {minlat},{maxlon} {minlat},{maxlon} {maxlat},{minlon} {maxlat},{minlon} {minlat}))'"
aoiQueryString  = f"OData.CSC.Intersects(area=geography'SRID=4326;{aoi})"
dateQueryString = f"ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T00:00:00.000Z"

nameQueryString = "startswith(Name, 'S2') and contains(Name, 'L2A')"


str = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter={nameQueryString} and {aoiQueryString} and {dateQueryString}"
print(str)

json = requests.get(
    f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter={nameQueryString} and {aoiQueryString} and {dateQueryString}"

).json()
pd.DataFrame.from_dict(json["value"]).head(5)

https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=startswith(Name, 'S2') and contains(Name, 'L2A') and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON((-3 52,-2 52,-2 53,-3 53,-3 52))') and ContentDate/Start gt 2023-12-01T00:00:00.000Z and ContentDate/Start lt 2024-05-10T00:00:00.000Z


,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,a0f2db1f-2903-413c-a44f-e742f0d5144f,S2A_MSIL2A_20240116T111411_N0510_R137_T30UWC_2...,application/octet-stream,1158557138,2024-01-16T16:21:21.000000Z,2024-01-16T16:33:59.345732Z,2024-03-13T18:31:11.225551Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2024/01/16/S2A_MSIL...,"[{'Value': '9ccffbaf5760ba68e1abdfb3dde62b07',...","{'Start': '2024-01-16T11:14:11.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-3.0002888420798...,"{'type': 'Polygon', 'coordinates': [[[-3.00028..."
1,application/octet-stream,2f9985a7-3007-4102-a081-64018b81bcf8,S2B_MSIL2A_20240101T111359_N0510_R137_T30UWD_2...,application/octet-stream,849947476,2024-01-01T13:54:57.372000Z,2024-01-01T14:09:11.460459Z,2024-03-13T13:34:16.680024Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2024/01/01/S2B_MSIL...,"[{'Value': '9cbc1cc133ca6da279977adc2d430f26',...","{'Start': '2024-01-01T11:13:59.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-2.7489816822225...,"{'type': 'Polygon', 'coordinates': [[[-2.74898..."
2,application/octet-stream,0e7fab72-65b9-4902-b0c1-10921d6c1d8c,S2B_MSIL2A_20240213T112119_N0510_R037_T30UVD_2...,application/octet-stream,876419185,2024-02-13T13:47:19.000000Z,2024-02-13T13:59:47.816268Z,2024-03-13T08:34:52.330087Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2024/02/13/S2B_MSIL...,"[{'Value': '5b21ae01b8d1cdc10d69128f87a8a380',...","{'Start': '2024-02-13T11:21:19.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-4.4990595879024...,"{'type': 'Polygon', 'coordinates': [[[-4.49905..."
3,application/octet-stream,69278917-1346-4e60-a416-31bc8250f0a8,S2B_MSIL2A_20240213T112119_N0510_R037_T30UVC_2...,application/octet-stream,801888529,2024-02-13T13:55:01.000000Z,2024-02-13T14:04:32.126956Z,2024-03-13T08:34:54.194663Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2024/02/13/S2B_MSIL...,"[{'Value': 'aa0e4c51d7a9705b74cb7a73166c1243',...","{'Start': '2024-02-13T11:21:19.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-4.4684710974117...,"{'type': 'Polygon', 'coordinates': [[[-4.46847..."
4,application/octet-stream,714745dc-bbf6-4264-a64b-922151783d9f,S2A_MSIL2A_20240208T112231_N0510_R037_T30UVC_2...,application/octet-stream,785250296,2024-02-08T15:40:14.000000Z,2024-02-08T15:51:44.595798Z,2024-03-13T10:15:16.852334Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2024/02/08/S2A_MSIL...,"[{'Value': '6161eac179233c12e13ae875523edc8d',...","{'Start': '2024-02-08T11:22:31.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-4.4684710974117...,"{'type': 'Polygon', 'coordinates': [[[-4.46847..."


In [27]:
url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products(7d874706-b8b3-4b55-a9f0-3e8d947bf91b)/$value"

headers = {"Authorization": f"Bearer {access_token}"}

session = requests.Session()
session.headers.update(headers)
response = session.get(url, headers=headers, stream=True)

with open("product.zip", "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)

In [47]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sentinelhub import SHConfig, BBox, CRS, SentinelHubRequest, MimeType, bbox_to_dimensions


#Client ID: sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9
#Client secret: BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5

# Configure your credentials
config = SHConfig()
config.sh_client_id = 'sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9'
config.sh_client_secret = 'BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5'
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'

# Define the area of interest
aoi_coords = [46.16, -16.15, 46.51, -15.58]  # Example coordinates
aoi_bbox = BBox(bbox=aoi_coords, crs=CRS.WGS84)
resolution = 10  # in meters
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

# Define the evalscript for true-color imagery
evalscript_true_color = """
//VERSION=3
function setup() {
  return {
    input: ["B04", "B03", "B02"],
    output: { bands: 3 }
  };
}

function evaluatePixel(sample) {
  return [sample.B04, sample.B03, sample.B02];
}
"""

# Create the request
request = SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A,
            time_interval=('2023-12-01', '2024-05-10')
        )
    ],
    responses=[
        SentinelHubRequest.output_response('default', MimeType.TIFF)
    ],
    bbox=aoi_bbox,
    size=aoi_size,
    config=config
)

# Execute the request and download the image
image = request.get_data()[0]

# Display the image
plt.imshow(np.clip(image * 2.5, 0, 1))
plt.show()


InvalidClientError: (invalid_client) Invalid client or Invalid client credentials

In [41]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Your client credentials
#Client ID: sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9
#Client secret: BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5
client_id = 'sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9'
client_secret = 'BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret, include_client_id=True)

# All requests using this session will have an access token automatically added
resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")
print(resp.content)

b'[]'


In [47]:
from PIL import Image
from io import BytesIO

evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B03", "B04", "SCL"],
    output: { bands: 3 },
  }
}

function evaluatePixel(sample) {
  if ([8, 9, 10].includes(sample.SCL)) {
    return [1, 0, 0]
  } else {
    return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02]
  }
}
"""

maxlat = 51
minlat = 50
minlon = -2
maxlon = -1

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [minlon, minlat, maxlon, maxlat],
        },
        "data": [
            {
                "type": "sentinel-2-l2a",
                "dataFilter": {
                    "timeRange": {
                        "from": "2024-05-19T00:00:00Z",
                        "to":   "2024-05-21T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 2048,
        "height": 2048,
        "responses": [
            {
                "identifier": "default",
                "format": {
                    "type": "image/jpeg",
                    "quality": 80,
                },
            }
        ],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Output the result to a file
if response.status_code == 200:
    image = Image.open(BytesIO(response.content))
    image.save("output_image.jpg")
else:
    print(f"Request failed with status code {response.status_code}")

